## ДЗ 1
#### Гордеев Никита

Нашим корпусом будет подборка новостных заметок rbc.ru: у них есть разметка ключевых слов в виде тегов для каждой новости.

Я взял 9 статей поразнообразней, получилось немного больше 3к токенов. Ключевые слова с РБК меня, в целом устроили, но я добавил свои и убрал некоторые малоинформативные (например, новости). Также в тегах-персонах для удобства я оставил только фамилии.

В итоге получился файл с ключевыми словами на нечётных строках и текстами заметок на чётных.

In [187]:
import RAKE
import nltk
import re
import pandas as pd

from keybert import KeyBERT
from nltk.corpus import stopwords
from summa import keywords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stop = stopwords.words('russian') + ['мочь', 'смочь', 'тема', 'иной', 'сказать', 'год', 'это', 'млн', 'млрд', 'который']

In [71]:
with open('nlp.txt', encoding='utf-8') as file:
    corpus = file.read().splitlines()

In [104]:
m = MorphAnalyzer()
def get_tokens(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return lemmas

In [219]:
text = corpus[1]

Мы будем использовать три модели: Rake, TextRank и KeyBERT.

### KeyBERT

In [222]:
#kw_model = KeyBERT()
kws = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2),
                                stop_words=stop, use_maxsum=True, nr_candidates=20, top_n=10)
kws

[('энергоресурсов россии', -0.0718),
 ('прогнозируют 2050', 0.1054),
 ('энергоперехода произойдет', 0.0065),
 ('национального рейтингового', 0.0282),
 ('готова возникающим', 0.04),
 ('устойчивого развития', 0.2333),
 ('энергоперехода российских', -0.0524)]

### RAKE

In [220]:
rake = RAKE.Rake(stop)
kws = rake.run(text, maxWords=3, minFrequency=2)
kws

[('углеродной нейтральности', 4.666666666666667),
 ('рынок труда', 4.0),
 ('россии', 1.6666666666666667),
 ('ведомстве', 1.0),
 ('таких', 1.0),
 ('энергопереход', 1.0),
 ('7', 0)]

### TextRank

In [221]:
kws = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)
kws

[('россии', 0.21464521153733043),
 ('россия', 0.21464521153733043),
 ('года', 0.20817758236574146),
 ('годы', 0.20817758236574146),
 ('году', 0.20817758236574146),
 ('годов', 0.20817758236574146),
 ('энергопереход', 0.20205184518407845),
 ('влияние', 0.13158208147988057),
 ('вызовов глобального энергоперехода', 0.13093682216750635),
 ('оценка влияния', 0.12216596454322767),
 ('энергетике', 0.11854817443237534),
 ('энергетики', 0.11854817443237534),
 ('развития', 0.11608648195129456),
 ('оценку', 0.11274984760657476),
 ('оценкам', 0.11274984760657476),
 ('доходы', 0.11242101208921138),
 ('доходов', 0.11242101208921138),
 ('дополнительных', 0.11043343483559864),
 ('зеленый', 0.10519253971414781),
 ('зеленого', 0.10519253971414781),
 ('российских', 0.10469243813891815),
 ('развитая атомная энергетика', 0.10396166266612107),
 ('странами', 0.10367323309460395),
 ('страной', 0.10367323309460395),
 ('страны', 0.10367323309460395),
 ('стран', 0.10367323309460395),
 ('федерального', 0.100982843

На первый взгляд KeyBERT, кажется, даёт наиболее релевантные результаты, хотя в целом, у всех моделей варианты не очень.
Посмотрим, что будет после лемматизации и с синтаксическими шаблонами.

In [179]:
def get_metrics(ref, ref2, kws, var):
    if var == 1:
        a = 0
        for kw in kws:
            if kw[0] in ref or kw[0] in ref2:
                a += 1
        if len(kws) == 0:
            precision = 0
            recall = 0
        else:
            precision = a / len(kws)
            recall = a / len(reference)
        if precision + recall == 0:
            f = 0
        else:
            f = precision * recall * 2 / (precision + recall)
    else:
        a = 0
        for kw in kws:
            if kw in ref or kw in ref2:
                a += 1
        if len(kws) == 0:
            precision = 0
            recall = 0
        else:
            precision = a / len(kws)
            recall = a / len(reference)
        if precision + recall == 0:
            f = 0
        else:
            f = precision * recall * 2 / (precision + recall)
    return round(precision, 4), round(recall, 4), round(f, 4)

In [223]:
strings = []
for j in range(len(corpus)):
    if j % 2 == 0:
        reference = corpus[j].lower().split(',')
        reference_lemmas = get_tokens(corpus[j])
    else:
        #собираем словосочетания
        text = re.sub('[0-9]', '', corpus[j])        
        tokens = get_tokens(text)
        text = ' '.join(tokens)
        poses = []
        filters = []
        for token in tokens:
            poses.append(m.parse(token)[0].tag.POS)
        for i in range(len(tokens[1:])):
            if poses[i] != None and poses[i+1] != None:
                colloc = poses[i] + poses[i+1]
                #берём прил + сущ и сущ + сущ
                if colloc == 'NOUNNOUN' or colloc == 'ADJFNOUN':
                    colloc = ' '.join([tokens[i], tokens[i+1]])
                    if colloc not in filters:
                        filters.append(colloc)
        #применяем модели
        kw_keybert = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1),
                                    stop_words=stop, use_maxsum=True, nr_candidates=20, top_n=10)
        kw_rake = rake.run(text, maxWords=3, minFrequency=2)
        kw_textrank = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)
        #с синтаксисом
        kw_keybert_synt = []
        for kw in kw_keybert:
            if kw[0] in filters or ' ' not in kw[0]:
                kw_keybert_synt.append(kw[0])
        kw_rake_synt = []
        for kw in kw_rake:
            if kw[0] in filters or ' ' not in kw[0]:
                kw_rake_synt.append(kw[0])
        kw_textrank_synt = []
        for kw in kw_textrank:
            if kw[0] in filters or ' ' not in kw[0]:
                kw_textrank_synt.append(kw[0])
        print(reference)
        #метрики
        string = []
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_keybert, 1)
        string.extend([precision, recall, f])
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_rake, 1)
        string.extend([precision, recall, f])
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_textrank, 1)
        string.extend([precision, recall, f])
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_keybert_synt, 2)
        string.extend([precision, recall, f])
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_rake_synt, 2)
        string.extend([precision, recall, f])
        precision, recall, f = get_metrics(reference, reference_lemmas, kw_textrank_synt, 2)
        string.extend([precision, recall, f])
        strings.append(string)

['счетная палата', 'нефть', 'газ', 'углеводороды', 'энергопереход', 'зеленый переход', 'экономика', 'экология', 'энергетика', 'россия', 'лес']
['linkedin', 'шпионаж', 'нидерланды', 'китай', 'россия', 'технологии', 'соцсети']
['хоккей', 'коронавирус', 'олимпиада', 'тест', 'маска', 'канада', 'россия']
['хованский', 'санкт-петербург', 'запрет', 'блогер', 'песня', 'терроризм']
['коронавирус', 'испания', 'спорт', 'ученые', 'ожирение', 'covid-19']
['рпц', 'владимир легойда', 'дискриминация', 'атеизм', 'церковь', 'ценности']
['tesla', 'авто', 'биткоин', 'сша', 'криптовалюта', 'актив']
['роскосмос', 'робот', 'космос']
['макрон', 'путин', 'переговоры', 'украина', 'визит', 'встреча', 'москва']


In [224]:
df = pd.DataFrame(strings, columns = ['prec_keybert', 'rec_keybert', 'f_keybert',
                             'prec_rake', 'rec_rake', 'f_rake',
                             'prec_tr', 'rec_tr', 'f_tr',
                             'prec_keybert_synt', 'rec_keybert_synt', 'f_keybert_synt',
                             'prec_rake_synt', 'rec_rake_synt', 'f_rake_synt',
                             'prec_tr_synt', 'rec_tr_synt', 'f_tr_synt'])

In [208]:
# keyBERT можно искать биграммы

,0,1,2,3,4,5,6,7,8
prec_keybert,0.0000,0.0000,0.0000,0.0000,0.0000,0.1429,0.0000,0.0000,0.0000
rec_keybert,0.0000,0.0000,0.0000,0.0000,0.0000,0.1667,0.0000,0.0000,0.0000
f_keybert,0.0000,0.0000,0.0000,0.0000,0.0000,0.1538,0.0000,0.0000,0.0000
prec_rake,0.2500,0.2857,0.6250,0.3333,0.2500,0.0000,0.2000,0.0000,0.0000
rec_rake,0.1818,0.2857,0.7143,0.1667,0.3333,0.0000,0.1667,0.0000,0.0000
f_rake,0.2105,0.2857,0.6667,0.2222,0.2857,0.0000,0.1818,0.0000,0.0000
prec_tr,0.0769,0.0769,0.1200,0.2000,0.0833,0.2000,0.2105,0.1538,0.2857
rec_tr,0.4545,0.4286,0.4286,0.5000,0.5000,0.6667,0.6667,0.6667,0.5714
f_tr,0.1316,0.1304,0.1875,0.2857,0.1429,0.3077,0.3200,0.2500,0.3810
prec_keybert_synt,0.0000,0.0000,0.0000,0.0000,0.0000,0.3333,0.0000,0.0000,0.0000


In [212]:
#keyBERT можно искать только униграммы

,0,1,2,3,4,5,6,7,8
prec_keybert,0.1429,0.0000,0.1429,0.2857,0.0000,0.1429,0.2857,0.0000,0.0000
rec_keybert,0.0909,0.0000,0.1429,0.3333,0.0000,0.1667,0.3333,0.0000,0.0000
f_keybert,0.1111,0.0000,0.1429,0.3077,0.0000,0.1538,0.3077,0.0000,0.0000
prec_rake,0.2500,0.2857,0.6250,0.3333,0.2500,0.0000,0.2000,0.0000,0.0000
rec_rake,0.1818,0.2857,0.7143,0.1667,0.3333,0.0000,0.1667,0.0000,0.0000
f_rake,0.2105,0.2857,0.6667,0.2222,0.2857,0.0000,0.1818,0.0000,0.0000
prec_tr,0.0769,0.0769,0.1200,0.2000,0.0833,0.2000,0.2105,0.1538,0.2857
rec_tr,0.4545,0.4286,0.4286,0.5000,0.5000,0.6667,0.6667,0.6667,0.5714
f_tr,0.1316,0.1304,0.1875,0.2857,0.1429,0.3077,0.3200,0.2500,0.3810
prec_keybert_synt,0.1429,0.0000,0.1429,0.2857,0.0000,0.1429,0.2857,0.0000,0.0000


In [225]:
df.transpose()

,0,1,2,3,4,5,6,7,8
prec_keybert,0.1000,0.1000,0.2000,0.2000,0.0000,0.2000,0.2000,0.1000,0.0000
rec_keybert,0.0909,0.1429,0.2857,0.3333,0.0000,0.3333,0.3333,0.3333,0.0000
f_keybert,0.0952,0.1176,0.2353,0.2500,0.0000,0.2500,0.2500,0.1538,0.0000
prec_rake,0.2500,0.2857,0.6250,0.3333,0.2500,0.0000,0.2000,0.0000,0.0000
rec_rake,0.1818,0.2857,0.7143,0.1667,0.3333,0.0000,0.1667,0.0000,0.0000
f_rake,0.2105,0.2857,0.6667,0.2222,0.2857,0.0000,0.1818,0.0000,0.0000
prec_tr,0.0769,0.0769,0.1200,0.2000,0.0833,0.2000,0.2105,0.1538,0.2857
rec_tr,0.4545,0.4286,0.4286,0.5000,0.5000,0.6667,0.6667,0.6667,0.5714
f_tr,0.1316,0.1304,0.1875,0.2857,0.1429,0.3077,0.3200,0.2500,0.3810
prec_keybert_synt,0.1000,0.1000,0.2000,0.2000,0.0000,0.2000,0.2000,0.1000,0.0000


Хуже всех справился KeyBERT: у него нулевые показатели во всех текстах, кроме одного. При разрешении n-грамм в ключевых словах он отдаёт предпочтение максимальному размеру n-граммы, который мы указали, от этого проседают сразу все метрики, потому что большинство ключевых слов в эталоне - униграммы. Если же ограничить поиск только униграммами, результаты получше, но на фоне конкурентов тоже не очень. Количество ключевых слов тоже опционально, но прямой корреляции метрик и количества нет. Единственный его плюс: он выдаёт топ ключевых слов даже для очень маленьких текстов.

TextRank берёт количеством: показатели полноты у него прекрасные, но и мусора много, оттого остальные показатели проседают.

Rake хорош, f-метрики у него самые лучшие, но в двух текстах он вообще не выделил ключевые слова, а в одном - только одно. К сожалению, модель справляется только с длинными текстами.

Фильтрация по синтаксическим паттернам помогла вырастить точность и соответственно f-score TextRank почти во текстах, для Rake помогла намного меньше, для KeyBERT вообще не помогла.